# Model Learning using $C^3$ with Tensorflow API

## Define a simple model

Load a one qubit device model from `.cfg` and `.hjsons`


## Run $C_1$ Optimal Control [Optional]

## Run $C_2$ Device Calibration

- Follow steps in `examples/Simulated_calibration.ipynb`
- Store data from the calibration run in a reusable format

The path to the dataset can be retrieved from opt.picklefilename

## Run $C_3$ Model Characterisation

- Load device model from configs
- Read data from the dataset stored in previous step
- Perform model learning with this data and model

In [1]:
import copy
import numpy as np
from c3.model import Model as Mdl
from c3.c3objs import Quantity as Qty
from c3.parametermap import ParameterMap as PMap
from c3.experiment import Experiment as Exp
from c3.generator.generator import Generator as Gnr
import c3.signal.gates as gates
import c3.libraries.chip as chip
import c3.generator.devices as devices
import c3.libraries.hamiltonians as hamiltonians
import c3.signal.pulse as pulse
import c3.libraries.envelopes as envelopes
import c3.libraries.tasks as tasks
from c3.utils import qt_utils

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
import pandas as pd
from pprint import pprint

In [4]:
c2_data = pd.read_pickle("dataset1.pkl")

### What does the dataset from $C^2$ look like?

In [5]:
c2_data.head()

,params,seqs,results,results_std,shots
0,"[(0.45), (-1.0), (-50500000.00000001), (4.0840...","[[rx90p[0], rx90p[0], ry90p[0], ry90p[0], ry90...","[[0.629], [0.837], [0.397], [0.696], [0.233], ...","[[0.015276092432294325], [0.011680368144883106...","[[1000], [1000], [1000], [1000], [1000], [1000..."
1,"[(0.4538241124244585), (-1.424521014737922), (...","[[ry90p[0], rx90p[0], ry90m[0], ry90m[0], rx90...","[[0.38], [0.905], [0.638], [0.881], [0.817], [...","[[0.015349267083479917], [0.00927227048785787]...","[[1000], [1000], [1000], [1000], [1000], [1000..."
2,"[(0.4169468405700841), (-1.0141995806928747), ...","[[rx90p[0], ry90m[0], rx90p[0], rx90p[0], rx90...","[[0.717], [0.945], [0.904], [0.745], [0.11], [...","[[0.014244683218660919], [0.007209368904418751...","[[1000], [1000], [1000], [1000], [1000], [1000..."
3,"[(0.4144062157834531), (-0.7321606412541017), ...","[[rx90p[0], ry90p[0], rx90p[0], ry90p[0], ry90...","[[0.328], [0.562], [0.727], [0.741], [0.723], ...","[[0.01484641370836742], [0.015689359451551872]...","[[1000], [1000], [1000], [1000], [1000], [1000..."
4,"[(0.40976180003577817), (-0.6025933839453428),...","[[ry90p[0], rx90p[0], ry90p[0], rx90m[0], rx90...","[[0.73], [0.098], [0.244], [0.67], [0.235], [0...","[[0.014039230748157109], [0.009401914698613257...","[[1000], [1000], [1000], [1000], [1000], [1000..."


In [6]:
c2_data['params'][0]

[450.000 mV, -1.000 , -50.500 MHz 2pi, 4.084 rad]

In [7]:
c2_data['params'][0][0]

450.000 mV

In [8]:
c2_data['params'][0]

[450.000 mV, -1.000 , -50.500 MHz 2pi, 4.084 rad]

In [9]:
lindblad = False
dressed = True
qubit_lvls = 3
freq = 5e9
anhar = -210e6
init_temp = 0
qubit_temp = 0
t_final = 7e-9  # Time for single qubit gates
sim_res = 100e9
awg_res = 2e9
sideband = 50e6
lo_freq = 5e9 + sideband

# ### MAKE GENERATOR
generator = Gnr(
    devices={
        "LO": devices.LO(name="lo", resolution=sim_res, outputs=1),
        "AWG": devices.AWG(name="awg", resolution=awg_res, outputs=1),
        "DigitalToAnalog": devices.DigitalToAnalog(
            name="dac", resolution=sim_res, inputs=1, outputs=1
        ),
        "Response": devices.Response(
            name="resp",
            rise_time=Qty(value=0.3e-9, min_val=0.05e-9, max_val=0.6e-9, unit="s"),
            resolution=sim_res,
            inputs=1,
            outputs=1,
        ),
        "Mixer": devices.Mixer(name="mixer", inputs=2, outputs=1),
        "VoltsToHertz": devices.VoltsToHertz(
            name="v_to_hz",
            V_to_Hz=Qty(value=1e9, min_val=0.9e9, max_val=1.1e9, unit="Hz/V"),
            inputs=1,
            outputs=1,
        ),
    },
    chains={
        "d1": ["LO", "AWG", "DigitalToAnalog", "Response", "Mixer", "VoltsToHertz"]
    },
)
generator.devices["AWG"].enable_drag_2()

# ### MAKE GATESET
gauss_params_single = {
    "amp": Qty(value=0.45, min_val=0.4, max_val=0.6, unit="V"),
    "t_final": Qty(
        value=t_final, min_val=0.5 * t_final, max_val=1.5 * t_final, unit="s"
    ),
    "sigma": Qty(
        value=t_final / 4, min_val=t_final / 8, max_val=t_final / 2, unit="s"
    ),
    "xy_angle": Qty(
        value=0.0, min_val=-0.5 * np.pi, max_val=2.5 * np.pi, unit="rad"
    ),
    "freq_offset": Qty(
        value=-sideband - 0.5e6,
        min_val=-60 * 1e6,
        max_val=-40 * 1e6,
        unit="Hz 2pi",
    ),
    "delta": Qty(value=-1, min_val=-5, max_val=3, unit=""),
}

gauss_env_single = pulse.Envelope(
    name="gauss",
    desc="Gaussian comp for single-qubit gates",
    params=gauss_params_single,
    shape=envelopes.gaussian_nonorm,
)
nodrive_env = pulse.Envelope(
    name="no_drive",
    params={
        "t_final": Qty(
            value=t_final, min_val=0.5 * t_final, max_val=1.5 * t_final, unit="s"
        )
    },
    shape=envelopes.no_drive,
)
carrier_parameters = {
    "freq": Qty(
        value=lo_freq,
        min_val=4.5e9,
        max_val=6e9,
        unit="Hz 2pi",
    ),
    "framechange": Qty(value=0.0, min_val=-np.pi, max_val=3 * np.pi, unit="rad"),
}
carr = pulse.Carrier(
    name="carrier",
    desc="Frequency of the local oscillator",
    params=carrier_parameters,
)

rx90p = gates.Instruction(
    name="rx90p", t_start=0.0, t_end=t_final, channels=["d1"], targets=[0]
)
QId = gates.Instruction(
    name="id", t_start=0.0, t_end=t_final, channels=["d1"], targets=[0]
)

rx90p.add_component(gauss_env_single, "d1")
rx90p.add_component(carr, "d1")
QId.add_component(nodrive_env, "d1")
QId.add_component(copy.deepcopy(carr), "d1")
QId.comps["d1"]["carrier"].params["framechange"].set_value(
    (-sideband * t_final) % (2 * np.pi)
)
ry90p = copy.deepcopy(rx90p)
ry90p.name = "ry90p"
rx90m = copy.deepcopy(rx90p)
rx90m.name = "rx90m"
ry90m = copy.deepcopy(rx90p)
ry90m.name = "ry90m"
ry90p.comps["d1"]["gauss"].params["xy_angle"].set_value(0.5 * np.pi)
rx90m.comps["d1"]["gauss"].params["xy_angle"].set_value(np.pi)
ry90m.comps["d1"]["gauss"].params["xy_angle"].set_value(1.5 * np.pi)

# MAKE OPT_MAP
opt_map =   [
    [
      ("rx90p[0]", "d1", "gauss", "amp"),
      ("ry90p[0]", "d1", "gauss", "amp"),
      ("rx90m[0]", "d1", "gauss", "amp"),
      ("ry90m[0]", "d1", "gauss", "amp")
    ],
    [
      ("rx90p[0]", "d1", "gauss", "delta"),
      ("ry90p[0]", "d1", "gauss", "delta"),
      ("rx90m[0]", "d1", "gauss", "delta"),
      ("ry90m[0]", "d1", "gauss", "delta")
    ],
    [
      ("rx90p[0]", "d1", "gauss", "freq_offset"),
      ("ry90p[0]", "d1", "gauss", "freq_offset"),
      ("rx90m[0]", "d1", "gauss", "freq_offset"),
      ("ry90m[0]", "d1", "gauss", "freq_offset")
    ],
    [
      ("id[0]", "d1", "carrier", "framechange")
    ]
  ]
logdir = "/tmp/c3logs/blackbox"
qubit_labels = {
      "excited" : [(1,), (2,)]
  }

RB_length = 100 # How long each sequence is
RB_number = 200  # How many sequences
shots = 1000    # How many averages per readout

################################
### Simulation specific part ###
################################

do_noise = False  # Whether to add artificial noise to the results

qubit_label = list(qubit_labels.keys())[0]
state_labels = qubit_labels[qubit_label]
state_label = [tuple(l) for l in state_labels]

# Creating the RB sequences #
seqs = qt_utils.single_length_RB(
        RB_number=RB_number, RB_length=RB_length, target=0
)


In [10]:
class QSimulator(keras.layers.Layer):
    def __init__(self):
        super(QSimulator, self).__init__()
        freq_init = tf.constant(value=0.5, dtype=tf.float64)

        self.freq = tf.Variable(
            initial_value=freq_init, trainable=True,
        )

    def call(self, inputs):
        # ### MAKE MODEL
        # Scale Frequency
        min_val=4.995e9
        max_val=5.005e9
        freq_scaled = (self.freq.numpy() * (max_val - min_val)) + min_val
        q1 = chip.Qubit(
            name="Q1",
            desc="Qubit 1",
            freq=Qty(
                value=freq_scaled,
                min_val=4.995e9,
                max_val=5.005e9,
                unit="Hz 2pi",
            ),
            anhar=Qty(
                value=anhar,
                min_val=-380e6,
                max_val=-120e6,
                unit="Hz 2pi",
            ),
            hilbert_dim=qubit_lvls,
            temp=Qty(value=qubit_temp, min_val=0.0, max_val=0.12, unit="K"),
        )

        drive = chip.Drive(
            name="d1",
            desc="Drive 1",
            comment="Drive line 1 on qubit 1",
            connected=["Q1"],
            hamiltonian_func=hamiltonians.x_drive,
        )
        phys_components = [q1]
        line_components = [drive]

        init_ground = tasks.InitialiseGround(
            init_temp=Qty(value=init_temp, min_val=-0.001, max_val=0.22, unit="K")
        )
        task_list = [init_ground]
        model = Mdl(phys_components, line_components, task_list)
        model.set_lindbladian(lindblad)
        model.set_dressed(dressed)

        parameter_map = PMap(
            instructions=[QId, rx90p, ry90p, rx90m, ry90m], model=model, generator=generator
        )

        # ### MAKE EXPERIMENT
        exp = Exp(pmap=parameter_map)

        # Make params from inputs
        amp = Qty(value=inputs[0].numpy(), min_val=0.4, max_val=0.6, unit="V")
        delta = Qty(value=inputs[1].numpy(), min_val=-5, max_val=3, unit="")
        freq_offset = Qty(
            value=inputs[2].numpy(),
            min_val=-60000000.0,
            max_val=-40000000.0,
            unit="Hz 2pi",
        )
        framechange = Qty(value=inputs[3].numpy(), min_val=-np.pi, max_val=3 * np.pi, unit="rad")

        params = [amp, delta, freq_offset, framechange]
        
        # Transmitting the parameters to the experiment #
        exp.pmap.set_parameters(params, opt_map)
        exp.set_opt_gates_seq(seqs)

        # Simulating the gates #
        U_dict = exp.compute_propagators()

        # Running the RB sequences and read-out the results #
        pops = exp.evaluate(seqs)
        pop1s, _ = exp.process(pops, labels=state_label)

        results = []
        results_std = []
        shots_nums = []

        # Collecting results and statistics, add noise #
        if do_noise:
            for p1 in pop1s:
                draws = tf.keras.backend.random_binomial(
                    [shots],
                    p=p1[0],
                    dtype=tf.float64,
                )
                results.append([np.mean(draws)])
                results_std.append([np.std(draws)/np.sqrt(shots)])
                shots_nums.append([shots])
        else:
            for p1 in pop1s:
                results.append(p1.numpy())
                results_std.append([0])
                shots_nums.append([shots])

        #######################################
        ### End of Simulation specific part ###
        #######################################

        goal = np.mean(results)
        return tf.constant(value=goal, dtype=tf.float64)

In [11]:
par_arr = [i.get_value() for i in c2_data['params'][0]]

In [12]:
c2_data['params'][0]

[450.000 mV, -1.000 , -50.500 MHz 2pi, 4.084 rad]

In [13]:
par_num = [tf.constant(value=i.numpy(), dtype=tf.float64) for i in c2_data['params'][0]]
par_num_tf = tf.concat(par_num, axis = 0)
par_num_tf

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([ 4.50000000e-01, -1.00000000e+00, -5.05000000e+07,  4.08407045e+00])>

In [14]:
par_tf = tf.concat(par_arr, axis=0)

In [15]:
par_tf

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([ 4.50000000e-01, -1.00000000e+00, -3.17300858e+08,  4.08407045e+00])>

In [16]:
qsim_layer = QSimulator()

In [17]:
out = qsim_layer(par_num_tf)

In [18]:
out

<tf.Tensor: shape=(), dtype=float64, numpy=0.6168894872576263>